In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(3)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [79]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   899 ps 	/   269.7 mm
Bob_T   :    30 ps 	/     9.0 mm
Bob_R   :  -607 ps 	/  -182.1 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    13 ps 	/     3.9 mm
Bob_T   :     8 ps 	/     2.4 mm
Bob_R   :    -6 ps 	/    -1.8 mm


In [66]:
KMC.rotate_simulataneously(0, 0)

In [65]:
KMC.rotate_simulataneously(0, 45)

In [136]:
n_values = 300
trace_width = 2
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

No such comm: 9d0bc7de081c4c1ab6455b60b36f457e
No such comm: 687b80de788e4b9d9725130e4adfbb09


In [139]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [138]:

KMC.rotate_simulataneously(45, 45, wait_for_completion=False)

In [10]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [140]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, 11.25, -11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9, debug=True)



corr[a,b] = 0.66147
	N[|T,T>]=  1002	(0.340)
	N[|T,R>]=   263	(0.089)
	N[|R,T>]=   236	(0.080)
	N[|R,R>]=  1447	(0.491)

corr[a,B] = 0.67678
	N[|T,T>]=   998	(0.329)
	N[|T,R>]=   250	(0.082)
	N[|R,T>]=   240	(0.079)
	N[|R,R>]=  1544	(0.509)

corr[A,b] = 0.53755
	N[|T,T>]=   779	(0.288)
	N[|T,R>]=   197	(0.073)
	N[|R,T>]=   428	(0.158)
	N[|R,R>]=  1299	(0.481)

corr[A,B] = -0.45673
	N[|T,T>]=   318	(0.123)
	N[|T,R>]=   809	(0.314)
	N[|R,T>]=  1068	(0.414)
	N[|R,R>]=   382	(0.148)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.3325299071609793


In [91]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.662 -0.664 -0.622  0.614]
	N[a , b ]=[ 881  249  248 1604]
	N[a , b ]=[ 182 1332 1191  347]
	N[a , b ]=[  98  679 1205  431]
	N[a , b ]=[ 496  157  311 1647]

corr[a,B] = [ 0.68  -0.674 -0.69   0.689]
	N[a , B ]=[ 902  238  228 1673]
	N[a , B ]=[ 140 1325 1173  337]
	N[a , B ]=[ 130  669 1256  311]
	N[a , B ]=[ 515  151  217 1848]

corr[A,b] = [ 0.434 -0.479 -0.444  0.368]
	N[A , b ]=[ 731  184  356 1328]
	N[A , b ]=[135 980 979 578]
	N[A , b ]=[ 349  824 1027  685]
	N[A , b ]=[ 495  452  416 1408]

corr[A,B] = [-0.538  0.532  0.45  -0.429]
	N[A , B ]=[ 288  810 1040  494]
	N[A , B ]=[ 560  382  369 1459]
	N[A , B ]=[ 790  183  400 1440]
	N[A , B ]=[ 117 1040  989  665]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.31432466 2.34834673 2.2065114  2.10005877]


In [97]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
TTC.measureS(CHSH_angles=johannes_classical_angles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiMinus, coincidence_window_SI=0.5e-9)


corr[a,b] = -1.0
	N[|T,T>]=     0	(0.000)
	N[|T,R>]=  2539	(0.508)
	N[|R,T>]=  2461	(0.492)
	N[|R,R>]=     0	(0.000)

corr[a,B] = 0.0136
	N[|T,T>]=  1277	(0.255)
	N[|T,R>]=  1234	(0.247)
	N[|R,T>]=  1232	(0.246)
	N[|R,R>]=  1257	(0.251)

corr[A,b] = -0.0068
	N[|T,T>]=  1249	(0.250)
	N[|T,R>]=  1258	(0.252)
	N[|R,T>]=  1259	(0.252)
	N[|R,R>]=  1234	(0.247)

corr[A,B] = 1.0
	N[|T,T>]=  2485	(0.497)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=  2515	(0.503)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 1.9932


# OSC Server Setup


In [7]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)
Received OSC_message: ('Q_all',)


In [8]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [9]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [12]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, -11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [25]:
Myself.client.send_message("/bruQner/set/state", 'C_single')

In [14]:
stop_performance()

performance stopped...


In [15]:
start_performance()

Exception in thread Thread-11 (performance_cycle):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\ghana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ghana\AppData\Local\Temp\ipykernel_11976\31319739.py", line 38, in performance_cycle
TypeError: unhashable type: 'list'
